In [ ]:
import re

In [ ]:
class Map:
    """Mapping from one type to another"""

    def __init__(self, mappings: dict, output: str):
        self.mappings = mappings
        self.output = output

    def convert(self, value):
        for source_range, diff in self.mappings.items():
            if value in source_range:
                return value + diff, self.output
        return value, self.output

    @classmethod
    def from_rules(cls, rules: list[str], output: str):
        mappings = dict()
        for rule in rules:
            d, s, r = map(int, rule.split(" "))
            mappings[range(s, s + r)] = d - s
        return cls(mappings, output)

In [ ]:
from collections import defaultdict

data = defaultdict(list)
with open("day05_input.txt") as file:
    for line in file:
        if match := re.match("seeds: (.+)", line):
            seeds = [int(s) for s in match.group(1).split(" ")]
        elif match := re.match("(\w+)-to-(\w+)", line):
            source, dest = match.groups()
        elif match := re.match("(\d+) (\d+) (\d+)", line):
            data[(source, dest)].append(line.strip())

In [ ]:
maps = dict()
for (source, dest), rules in data.items():
    maps[source] = Map.from_rules(rules, dest)

# Part 1


In [ ]:
locations = list()

for value in seeds:
    output = "seed"
    while True:
        value, output = maps[output].convert(value)
        # print(output, value)
        if not output in maps:
            locations.append(value)
            break

print("Answer:", min(locations))

# Part 2


In [ ]:
import re
import portion as P

In [ ]:
# Read the file
with open("day05_input.txt") as file:
    seeds = [int(s) for s in re.findall(r"\d+", file.readline())]
    blocks = file.read().split("\n\n")

# Find the initial ranges from the seeds
ranges = P.empty()
for start, length in zip(seeds[::2], seeds[1::2]):
    ranges = ranges | P.closedopen(start, start + length)

# Process the blocks with mappings
for block in blocks:
    remove, add = P.empty(), P.empty()
    for line in block.strip().split("\n")[1:]:
        dstart, sstart, length = map(int, line.split(" "))
        intersects = ranges & P.closedopen(sstart, sstart + length)
        shift = dstart - sstart
        new = intersects.apply(
            lambda x: x.replace(lower=x.lower + shift, upper=x.upper + shift)
        )
        remove = remove | intersects
        add = add | new
    ranges = ranges - remove | add

print("Answer:", ranges.lower)